In [ ]:
"""
Created on Wed Jun 09 14:36 2021

Prepare bedrock slope for use in the neural network

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm

In [ ]:
nemo_run = 'OPM016'
inputpath_data='/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

In [ ]:
def cut_domain_stereo(var_to_cut, map_lim_x, map_lim_y):
    var_cutted = var_to_cut.sel(x=var_to_cut.x.where(in_range(var_to_cut.x,map_lim_x),drop=True), y=var_to_cut.y.where(in_range(var_to_cut.y,map_lim_y),drop=True))
    return var_cutted

def in_range(in_xy,txy):
    return ((in_xy >= min(txy)) & (in_xy < max(txy)))

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
map_lim = [-3000000,3000000]
file_mask_orig = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')
file_mask_orig_cut = cut_domain_stereo(file_mask_orig, map_lim, map_lim)
file_other = xr.open_dataset(inputpath_data+'corrected_draft_bathy_isf.nc')#, chunks={'x': chunk_size, 'y': chunk_size})
file_other_cut = cut_domain_stereo(file_other, map_lim, map_lim)

In [ ]:
#file_bed_orig = file_other_cut['corrected_isf_bathy']
file_bed_orig = file_mask_orig_cut['bathy_metry']
file_draft = file_other_cut['corrected_isfdraft'] 

In [ ]:
file_other_cut

In [ ]:
file_draft.plot()

In [ ]:
def check_slope_one_dimension(input_da, shifted_plus, shifted_minus, dx):

    """
    Compute the basal slope at each point.
        
    Parameters
    ----------
    input_da : xr.DataArray
        Array where slope needs to be checked. For example: ice draft.
    shifted_plus : xr.DataArray
        Shifted version (positive direction) of input_da.
    shifted_minus : xr.DataArray
        Shifted version (negative direction) of input_da.
    dx : float
        Step in the coordinate along which input_da was shifted

    Returns
    -------
    slope: xr.DataArray
        slope along that coordinate, is 0 if nan
    """
    
    # check the direction in both dim directions
    slope_both = (shifted_minus - shifted_plus) / np.sqrt((2 * dx) ** 2)
    # if x+1 is nan, only take x - (x-1)
    slope_right = (input_da - shifted_plus) / np.sqrt(dx ** 2)
    # if x-1 is nan, only take x+1 - x
    slope_left = (shifted_minus - input_da) / np.sqrt(dx ** 2)
    # combine all of the above
    slope = slope_both.combine_first(slope_right).combine_first(slope_left)
    # set rest to 0
    slope = slope.where(np.isfinite(slope), 0)
    return slope


def compute_alpha_appenB(kisf, plume_var_of_int, ice_draft_neg, dx, dy):   

    """
    Compute alphas like in Appendix B of Favier et al., 2019 TCDiscussions.
    
    Parameters
    ----------
    kisf : int
        ID of the ice shelf of interest
    plume_var_of_int : xr.Dataset
        Dataset containing ``'ISF_mask'`` and ``'dIF'``
    ice_draft_neg : xr.DataArray
        Ice draft depth in m. Negative downwards.
    dx : float
        Grid spacing in the x-direction
    dy : float
        Grid spacing in the y-direction
        
    Returns
    -------
    go_back_to_whole_grid_local_alpha: xr.DataArray
        Local slope angle in rad for each point.
    """
    
    # cut out the ice shelf of interest
    draft_isf = ice_draft_neg.where(plume_var_of_int['ISF_mask'] == kisf, drop=True)

    shiftedx_minus = draft_isf.shift(x=-1)
    shiftedx_plus = draft_isf.shift(x=1)
    xslope = check_slope_one_dimension(draft_isf, shiftedx_plus, shiftedx_minus, dx)

    shiftedy_minus = draft_isf.shift(y=-1)
    shiftedy_plus = draft_isf.shift(y=1)
    yslope = check_slope_one_dimension(draft_isf, shiftedy_plus, shiftedy_minus, dy)

    dIF_isf = plume_var_of_int['dIF'].where(plume_var_of_int['ISF_mask'] == kisf)
    dIF_isf_corr = dIF_isf.where(dIF_isf/2500 < 1,1) #check again with Nico, if I understood it right (MIN to avoid strong frontal slope)

    local_alpha = np.arctan(np.sqrt(xslope ** 2 + yslope ** 2)) * dIF_isf_corr

    go_back_to_whole_grid_local_alpha = local_alpha.reindex_like(plume_var_of_int['ISF_mask'])

    return go_back_to_whole_grid_local_alpha

In [ ]:
dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]

bedrock_slope = None
for kisf in tqdm(file_isf.Nisf):
    #print(kisf.values)
    bb_sl = compute_alpha_appenB(kisf, file_isf, -1*file_bed_orig, abs(dx), abs(dy))
    if bedrock_slope is None:
        bedrock_slope = bb_sl
    else:
        bedrock_slope = bedrock_slope.combine_first(bb_sl)
bedrock_slope_smooth = bedrock_slope.reindex_like(file_isf)
bedrock_slope_smooth.to_dataset(name='bedrock_slope').to_netcdf(inputpath_mask+'nemo_5km_bedrock_slope.nc','w')

In [ ]:
def slope_in_x_and_y_dir(kisf, plume_var_of_int, ice_draft_neg, dx, dy):
    # cut out the ice shelf of interest
    draft_isf = ice_draft_neg.where(plume_var_of_int['ISF_mask'] == kisf, drop=True)
    
    shiftedx_minus = draft_isf.shift(x=-1)
    shiftedx_plus = draft_isf.shift(x=1)
    xslope = check_slope_one_dimension(draft_isf, shiftedx_plus, shiftedx_minus, dx)

    shiftedy_minus = draft_isf.shift(y=-1)
    shiftedy_plus = draft_isf.shift(y=1)
    yslope = check_slope_one_dimension(draft_isf, shiftedy_plus, shiftedy_minus, dy)
    
    xslope_whole_grid = xslope.reindex_like(plume_var_of_int['ISF_mask'])
    yslope_whole_grid = yslope.reindex_like(plume_var_of_int['ISF_mask'])
    
    return xslope_whole_grid, yslope_whole_grid

In [ ]:
dx = file_isf.x[2] - file_isf.x[1]
dy = file_isf.y[2] - file_isf.y[1]

x_slope_ice = None
y_slope_ice = None
x_slope_bed = None
y_slope_bed = None

for kisf in tqdm(file_isf.Nisf.sel(Nisf=[66])):
    #print(kisf.values)
    bb_sl_x, bb_sl_y = slope_in_x_and_y_dir(kisf, file_isf, -1*file_bed_orig, abs(dx), abs(dy))
    ice_sl_x, ice_sl_y = slope_in_x_and_y_dir(kisf, file_isf, -1*file_draft, abs(dx), abs(dy))
    if x_slope_bed is None:
        x_slope_bed = bb_sl_x
        y_slope_bed = bb_sl_y
        x_slope_ice = ice_sl_x
        y_slope_ice = ice_sl_y
    else:
        x_slope_bed = x_slope_bed.combine_first(bb_sl_x)
        y_slope_bed = y_slope_bed.combine_first(bb_sl_y)
        x_slope_ice = x_slope_ice.combine_first(ice_sl_x)
        y_slope_ice = y_slope_ice.combine_first(ice_sl_y)
        
x_slope_bed_smooth = x_slope_bed.reindex_like(file_isf)
y_slope_bed_smooth = y_slope_bed.reindex_like(file_isf)
x_slope_ice_smooth = x_slope_ice.reindex_like(file_isf)
y_slope_ice_smooth = y_slope_ice.reindex_like(file_isf)

#bedrock_slope_smooth.to_dataset(name='bedrock_slope').to_netcdf(inputpath_mask+'nemo_5km_bedrock_slope.nc','w')

In [ ]:
y_slope_bed.where(file_isf['ISF_mask']==66, drop=True).plot()

In [ ]:
file_bed_orig.where(file_isf['ISF_mask']==66, drop=True).plot()